In [ ]:
from pathlib import Path
import json
import pyarrow
import math
import re
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

from scipy.stats import gaussian_kde
from scipy.ndimage import label
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import holoviews as hv
from holoviews import opts

import traceback


hv.extension("bokeh")

from shiny import App, Inputs, Outputs, Session, reactive, render, req, ui

from utils_behavior import (
    Sleap_utils,
    HoloviewsTemplates,
    Utils,
    Processing,
    Ballpushing_utils,
    Seaborn_Templates,
)

import Config

import importlib

In [ ]:
importlib.reload(Config)

In [ ]:
BallTrajectories = pd.read_feather(
    "/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Coordinates/240110_coordinates_Data/coordinates/250106_Pooled_coordinates.feather"
)

In [ ]:
importlib.reload(Config)

In [ ]:
BallTrajectories = Config.cleanup_data(BallTrajectories)

BallTrajectories = Config.map_split_registry(BallTrajectories)

BallTrajectories

# Plot one particular Nickname

In [ ]:
MBON08 = BallTrajectories[BallTrajectories["Genotype"] == "TNTxG87"]

In [ ]:
MBON08["fly"].unique()

In [ ]:
importlib.reload(Config)

In [ ]:
subset = Config.get_subset_data(BallTrajectories,"Nickname", "SS54549 (P-F3-5R patch line)")

subset

In [ ]:
subset.groupby("Nickname").apply(lambda x: x["fly"].nunique())

In [ ]:
# Check the distribution of frame values grouped by Nickname

subset.groupby("Nickname")["frame"].describe()

In [ ]:
# Check how many unique "fly " values I have grouped by Nickname

subset.groupby("Nickname").apply(lambda x: x["fly"].nunique())

In [ ]:
importlib.reload(Processing)

In [ ]:
# Apply some lowpass filtering to distance_ball_0 grouped by fly

for fly in subset["fly"].unique():
    subset.loc[subset["fly"] == fly, "distance_ball_0_filtered"] = Processing.savgol_lowpass_filter(
        subset.loc[subset["fly"] == fly, "distance_ball_0"], 22
    )

In [ ]:
subset

In [ ]:
# Plot distance_ball_0 as function of time hue by Brain region

plt.figure(figsize=(10, 6))

sns.lineplot(
    data=subset,
    x="time",
    y="distance_ball_0",
    hue="Brain region",
    ci=None,
    palette=Config.color_dict,
)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
# Plot only one nickname but plot each individual fly separately

plt.figure(figsize=(10, 6))

sns.lineplot(
    data=subset[subset["Nickname"] == "MBON-08-GaL4  MBON-09-GaL4 "],
    x="time",
    y="distance_ball_0",
    hue="fly",
    ci=None,
)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

# Plot all nicknames groupes by brain region

In [ ]:
importlib.reload(Config)
importlib.reload(Processing)

In [ ]:
# Initialize random generator
rg = np.random.default_rng()

output_dir = Path("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250130_Coordinates_Downsampled_Signif_2/")

# If output directory does not exist, create it

output_dir.mkdir(parents=True, exist_ok=True)

# Process brain regions
miscellaneous_nicknames = []
for brain_region in Config.registries["brain_regions"]:
    output_path = output_dir / f"Full_euclidean_distance_coordinates_line_{brain_region}.png"
    
    if os.path.exists(output_path):
        print(f"Skipping Brain region {brain_region} as the plot already exists.")
        continue
    
    region_data = BallTrajectories[BallTrajectories['Brain region'] == brain_region]
    nicknames = region_data['Nickname'].unique()
    
    if len(nicknames) < 5 and brain_region != "Control":
        miscellaneous_nicknames.extend(nicknames)
        continue
    
    if brain_region != "Control":
        try:
            Config.create_and_save_plot(BallTrajectories, nicknames, brain_region, output_path, Config.registries, show_signif=True)
            print(f"Processed Brain region {brain_region}")
        except Exception as e:
            print(f"Error processing Brain region {brain_region}: {e}")
            traceback.print_exc()

# Process Miscellaneous brain region
if miscellaneous_nicknames:
    try:
        output_path = output_dir / "Full_euclidean_distance_coordinates_line_Miscellaneous.png"
        Config.create_and_save_plot(BallTrajectories, miscellaneous_nicknames, "Miscellaneous", output_path, Config.registries, show_signif=True)
        print("Processed Miscellaneous Brain region")
    except Exception as e:
        print(f"Error processing Miscellaneous Brain region: {e}")
        traceback.print_exc()

# Plot the Control brain region
try:
    control_nicknames = Config.registries["control_nicknames"]
    output_path = output_dir / "Full_euclidean_distance_coordinates_line_Control.png"
    Config.create_control_plot(BallTrajectories, control_nicknames, output_path)
    print("Processed Control Brain region")
except Exception as e:
    print(f"Error processing Control Brain region: {e}")
    traceback.print_exc()

# Load data from individual datasets

In [ ]:
importlib.reload(Config)

In [ ]:
# Process brain regions

registries = Config.registries
data_path = Path("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/240120_short_contacts_no_cutoff_no_downsample_Data/coordinates_regions")
miscellaneous_nicknames = []
for brain_region in registries["brain_regions"]:
    
    if brain_region != "Control":
        output_path = f"/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250122_Coordinates_full/Full_euclidean_distance_coordinates_line_{brain_region}.png"
    
        if os.path.exists(output_path):
            print(f"Skipping Brain region {brain_region} as the plot already exists.")
            continue
        
        BallTrajectories = Config.load_datasets_for_brain_region(brain_region, data_path, registries, downsample_factor=10)
        
        region_data = BallTrajectories[BallTrajectories['Brain region'] != "Control"]
        nicknames = region_data['Nickname'].unique()
        
        if len(nicknames) < 5 and brain_region != "Control":
            miscellaneous_nicknames.extend(nicknames)
            continue
        try:
            Config.create_and_save_plot(region_data, nicknames, brain_region, output_path, registries, show_signif=True)
            print(f"Processed Brain region {brain_region}")
        except Exception as e:
            print(f"Error processing Brain region {brain_region}: {e}")
            
    else:
        output_path = "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250120_coordinates/Full_euclidean_distance_coordinates_line_Control.png"
        
        if os.path.exists(output_path):
            
            print(f"Skipping Control Brain region as the plot already exists.")
            continue
        
        BallTrajectories = Config.load_datasets_for_brain_region(brain_region, data_path, registries, downsample_factor=10)


        nicknames = region_data['Nickname'].unique()
        
        # Plot the Control brain region
        try:
            control_nicknames = registries["control_nicknames"]
            
            Config.create_control_plot(region_data, control_nicknames, output_path)
            print("Processed Control Brain region")
        except Exception as e:
            print(f"Error processing Control Brain region: {e}")

# Process Miscellaneous brain region
if miscellaneous_nicknames:
    try:
        output_path = "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250120_coordinates/Full_euclidean_distance_coordinates_line_Miscellaneous.png"
        Config.create_and_save_plot(region_data, miscellaneous_nicknames, "Miscellaneous", output_path, registries, show_signif=True)
        print("Processed Miscellaneous Brain region")
    except Exception as e:
        print(f"Error processing Miscellaneous Brain region: {e}")



# Permutation test

In [ ]:
CX = pd.read_feather("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/240120_short_contacts_no_cutoff_no_downsample_Data/coordinates_regions/CX.feather")

In [ ]:
control = pd.read_feather("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/240120_short_contacts_no_cutoff_no_downsample_Data/coordinates_regions/Control.feather")

In [ ]:
CX_ctrl = pd.concat([CX, control], ignore_index=True)

In [ ]:
# Pick a random subset

Test = Config.get_subset_data(CX_ctrl, "Nickname", "random")

In [ ]:
Test.head()

In [ ]:
# Split the data by Nickname

Focal = Test[Test["Brain region"] != "Control"]

Control = Test[Test["Brain region"] == "Control"]

In [ ]:
# Make the average distance_ball_0 grouped by time

Focal_avg = Focal["distance_ball_0"].groupby(Focal["time"]).mean()

Ctrl_avg = Control["distance_ball_0"].groupby(Control["time"]).mean()

In [ ]:
# Set the column "time" as index

Focal_avg = Focal_avg.reset_index()

Ctrl_avg = Ctrl_avg.reset_index()

In [ ]:
# Step 2: Ensure both DataFrames have the same index (time points)
common_index = Focal_avg.index.intersection(Ctrl_avg.index)
Focal_avg = Focal_avg.loc[common_index]
Ctrl_avg = Ctrl_avg.loc[common_index]

In [ ]:
# Step 3: Call the permutation_test function
observed_diff, p_values = Processing.permutation_test(Focal_avg, Ctrl_avg, n_permutations=1000)


In [ ]:
# Find significant time points
significance_level = 0.05
significant_timepoints = np.where(p_values < significance_level)[0]

print(f"Number of significant time points: {len(significant_timepoints)}")
print(f"Percentage of significant time points: {len(significant_timepoints) / len(p_values) * 100:.2f}%")

# Optional: Apply multiple testing correction
from statsmodels.stats.multitest import multipletests

rejected, p_values_corrected, _, _ = multipletests(p_values, method='fdr_bh')
significant_timepoints_corrected = np.where(rejected)[0]

print(f"Number of significant time points after correction: {len(significant_timepoints_corrected)}")
print(f"Percentage of significant time points after correction: {len(significant_timepoints_corrected) / len(p_values) * 100:.2f}%")

In [ ]:

plt.figure(figsize=(12, 6))
plt.plot(observed_diff.index, observed_diff.values, label='Observed Difference')
plt.scatter(observed_diff.index[significant_timepoints], 
            observed_diff.values[significant_timepoints], 
            color='red', label='Significant Points')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Difference in distance_ball_0')
plt.title('Permutation Test Results: Focal vs Control')
plt.legend()
plt.show()

In [ ]:
# Plot the trajectories associated with each Nickname

sns.lineplot(data=Test, x="time", y="distance_ball_0", hue="Nickname", ci=None)

In [ ]:
# Add a column to indicate significant timepoints
Test['Significant'] = Test['time'].isin(Focal_avg.iloc[significant_timepoints]['time'])

In [ ]:
importlib.reload(Config)

In [ ]:
significant_times = Test[Test['Significant']]['time']
            

In [ ]:
sns.lineplot(data=Test, x='time', y='distance_ball_0', hue='Brain region', palette=Config.color_dict, ci=None)

for time in significant_times:
    plt.axvspan(time - 0.5, time + 0.5, color='red', alpha=0.3)
    
    plt.set_title(f'Test')
    plt.set_xlabel('Time (s)')
    plt.set_ylabel('Median Euclidean Distance')
    